In [26]:
import requests
import random
from time import sleep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.cluster.vq import whiten, kmeans2

%matplotlib inline

In [20]:
def call_api(uri, result_key):
    json = {}
    try:
        json = requests.get(uri, headers = { 'X-Auth-Token': '4e6def2863544dd68e99c8f13f68d246' }).json()
        return json[result_key]
    except:
        print(json)
        return None

In [18]:
# Champions League 2016/17
teams = call_api('http://api.football-data.org/v1/soccerseasons/440/teams', 'teams')
teams

[{'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/729/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/729/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/729'}},
  'code': None,
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/commons/c/c5/FC_Basel.png',
  'name': 'FC Basel',
  'shortName': 'Basel',
  'squadMarketValue': None},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/753/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/753/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/753'}},
  'code': 'LUD',
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/commons/8/87/Ludogorets_FC.png',
  'name': 'Ludogorez Rasgrad',
  'shortName': 'Ludogorez',
  'squadMarketValue': None},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/5/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/5/players'},
   'self'

In [43]:
for team in teams:
    sleep(5) # 50 requests per minute are available only, so have to sleep

    players = call_api(team['_links']['players']['href'], 'players')
    fixtures = call_api(team['_links']['fixtures']['href'], 'fixtures')

    games = len(fixtures)
    loses = 0

    for f in fixtures:
        if team['name'] == f['homeTeamName']:
            loses += 1 if int(f['result']['goalsHomeTeam']) < int(f['result']['goalsAwayTeam']) else 0
        else:
            loses += 1 if int(f['result']['goalsAwayTeam']) < int(f['result']['goalsHomeTeam']) else 0

    market_value = team['squadMarketValue']
    if market_value is None:
        market_value = random.randint(1576, 3690) * (10 ** 6)

    print('Team "{}" costs {}, played {} games, had {} loses'.format(team['name'], market_value, games, loses))

    team['players'] = players
    team['cost'] = market_value
    team['games'] = games
    team['loses'] = loses

Team "FC Basel" costs 2008000000, played 8 games, had 3 loses
Team "Ludogorez Rasgrad" costs 1962000000, played 1 games, had 1 loses
Team "FC Bayern München" costs 1906000000, played 52 games, had 7 loses
Team "FC Rostov" costs 3432000000, played 0 games, had 0 loses
Team "PSV Eindhoven" costs 2146000000, played 34 games, had 3 loses
Team "Club Atlético de Madrid" costs 2677000000, played 44 games, had 6 loses
Team "FC Barcelona" costs 2815000000, played 48 games, had 2 loses
Team "Celtic FC" costs 1875000000, played 10 games, had 6 loses
Team "Dynamo Kyiv" costs 2049000000, played 0 games, had 0 loses
Team "SSC Napoli" costs 2032000000, played 46 games, had 7 loses
Team "SL Benfica" costs 2105000000, played 40 games, had 9 loses
Team "Besiktas JK" costs 3573000000, played 8 games, had 2 loses
Team "Paris Saint-Germain" costs 2437000000, played 46 games, had 6 loses
Team "Arsenal FC" costs 3004000000, played 38 games, had 13 loses
Team "Legia Warszawa" costs 3019000000, played 2 games,